In [3]:
# start SparkSession
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("project").getOrCreate()

In [4]:
#1. read in the merged dataset
data_df = spark.read\
  .format('csv')\
  .option('header', 'true')\
  .option('inferSchema', 'true')\
  .load('s3://bigdataproject233/millions.csv')

In [6]:
# data_df.show()

+--------------+-----+------------+--------------------+--------------------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+--------------------+--------------------+
|            by|score|        time|           timestamp|               title|   type|                 url|                text|              parent|             deleted|                dead|     descendants|                  id|             ranking|
+--------------+-----+------------+--------------------+--------------------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+--------------------+--------------------+
|         malyk| null|1511987746.0|2017-11-29 20:35:...|                null|comment|                null|That&#x27;s likel...|          15810293.0|                null|                null|            null|            15810594|                null|


In [5]:
data_df.count()

5642724

In [6]:
from pyspark.sql.types import IntegerType
from pyspark.sql.types import DoubleType

df_total = data_df.select(data_df['by'],
                     data_df['score'].cast(IntegerType()),
                     #data_df["time"],
                     data_df['timestamp'].cast('timestamp'),
                     data_df['title'],
                     data_df['type'],
                     data_df['url'],
                     data_df['text'],
                     data_df['parent'].cast(IntegerType()),
                     #data_df['deleted'],
                     #data_df['dead'],
                     data_df['descendants'].cast(IntegerType()),
                     data_df['id'].cast(IntegerType()),
                     data_df['ranking'].cast(IntegerType())
                             )


In [7]:
df_total.printSchema()

root
 |-- by: string (nullable = true)
 |-- score: integer (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- title: string (nullable = true)
 |-- type: string (nullable = true)
 |-- url: string (nullable = true)
 |-- text: string (nullable = true)
 |-- parent: integer (nullable = true)
 |-- descendants: integer (nullable = true)
 |-- id: integer (nullable = true)
 |-- ranking: integer (nullable = true)



In [8]:
df_total.show(10)

+-----------+-----+-------------------+-----+-------+----+--------------------+--------+-----------+--------+-------+
|         by|score|          timestamp|title|   type| url|                text|  parent|descendants|      id|ranking|
+-----------+-----+-------------------+-----+-------+----+--------------------+--------+-----------+--------+-------+
|      malyk| null|2017-11-29 20:35:46| null|comment|null|That&#x27;s likel...|15810293|       null|15810594|   null|
|   wbracken| null|2011-10-11 21:56:08| null|comment|null|Been wondering th...| 3100461|       null| 3100489|   null|
|      spang| null|2014-10-01 18:51:56| null|comment|null|"Inbox | <a href=...|    null|       null|    null|   null|
|        nfm| null|2015-11-14 22:25:35| null|comment|null|Yes, yes it is.<p...|10567471|       null|10567544|   null|
|         Mz| null|2010-10-17 20:45:41| null|comment|null|Something he does...| 1800936|       null| 1801375|   null|
|white-flame| null|2017-10-23 03:55:44| null|comment|nul

In [9]:
data = df_total.createOrReplaceTempView('data')

In [10]:
# Filter Story data 
story = spark.sql('SELECT * FROM data WHERE type == "story"')
story.createOrReplaceTempView('story')

In [11]:
story.count()

842645

In [20]:
story.show(50)

+---------------+-----+-------------------+--------------------+-----+--------------------+--------------------+------+-----------+--------+-------+
|             by|score|          timestamp|               title| type|                 url|                text|parent|descendants|      id|ranking|
+---------------+-----+-------------------+--------------------+-----+--------------------+--------------------+------+-----------+--------+-------+
|         yopiti|    1|2013-04-15 13:22:49|Why disabling 3rd...|story|http://www.spamco...|                null|  null|          0| 5551313|   null|
|   mikethecoach|    1|2017-12-07 19:11:41|How trainers and ...|story|https://www.podbe...|                null|  null|       null|15873005|   null|
|           frik|    2|2016-01-23 11:00:22|AppDynamics CEO: ...|story|http://venturebea...|                null|  null|          0|10957977|   null|
|  athinggoingon|    2|2017-11-30 22:08:19|Regulation free z...|story|http://www.nextgo...|               

In [13]:
story.select("id").distinct().count()

4041

In [16]:
story.select("title").distinct().count()

3890

In [17]:
story.select("timestamp").distinct().count()

4073

In [16]:
title_df = spark.sql('select * from story where title is not NULL')

In [13]:
title_df.count()

797343

In [17]:
title_df.createOrReplaceTempView('title')

In [18]:
# check if there is score as NULL in the dataset where title is not NULL
spark.sql('select score from title where score is not NULL').count()

797343

In [42]:
#待完成
#url_df = spark.sql("select score, by as author, title, regexp_extract(url, '/(\w+):\/\/([^/:]+)(:\d*)?([^# ]*)/', 1) as url from title")

In [43]:
#url_df.select('url').show(2)

+---+
|url|
+---+
|   |
|   |
+---+
only showing top 2 rows



In [60]:
#score information
spark.sql("select min(score),max(score),avg(score) from title").collect()

[Row(min(score)=0, max(score)=1129, avg(score)=11.096693643764352)]

In [67]:
# author/user information 
# decending by total_score 
sqlStatement = """SELECT SUM(score) as total_score, by as author,
SUM(descendants) as total_decendants,
count(id) as total_stories
FROM title
GROUP BY author
ORDER BY total_score DESC
LIMIT 20
"""
user_df = spark.sql(sqlStatement)
user_df.createOrReplaceTempView("user")

In [68]:
user_df.show(10)

+-----------+---------------+----------------+-------------+
|total_score|         author|total_decendants|total_stories|
+-----------+---------------+----------------+-------------+
|     231445|        vassvdm|          164410|          205|
|     201925|    aarohmankad|           37310|          205|
|     149035|       brisance|           66010|          205|
|     131200|thereare5lights|           60270|          205|
|     124454|           bane|           31416|         1639|
|     123615|       Artemis2|           31775|          410|
|     123410|         Ours90|           39975|          205|
|     100040|            jgv|           12710|          205|
|      94915|       krausejj|           80975|          410|
|      86305|       riledhel|           39770|          205|
+-----------+---------------+----------------+-------------+
only showing top 10 rows



In [70]:
# author/user information 
# decending by total_stories 
sqlStatement = """SELECT SUM(score) as total_score, by as author,
SUM(descendants) as total_decendants,
count(id) as total_stories
FROM title
GROUP BY author
ORDER BY total_stories DESC
LIMIT 20
"""
user_df2 = spark.sql(sqlStatement)

In [71]:
user_df2.show(10)

+-----------+----------+----------------+-------------+
|total_score|    author|total_decendants|total_stories|
+-----------+----------+----------------+-------------+
|      12915|   rbanffy|            3690|         5535|
|      49405|     ingve|           21935|         2460|
|      25415|     evo_9|            5124|         2459|
|      39770|prostoalex|           36900|         2255|
|       4509|   jonbaer|             410|         2254|
|      11070|  jseliger|            4100|         2050|
|       9425|shawndumas|            1637|         2049|
|       2870|      cwan|             205|         1845|
|      13320|   dnetesn|           11070|         1844|
|      30135|    uptown|           47150|         1640|
+-----------+----------+----------------+-------------+
only showing top 10 rows



In [72]:
# author/user information 
# decending by avg_score 
sqlStatement = """SELECT SUM(score) as total_score, by as author,
SUM(descendants) as total_decendants, SUM(score)/count(id) as avg_score,
count(id) as total_stories
FROM title
GROUP BY author
ORDER BY avg_score DESC
LIMIT 20
"""
user_df3 = spark.sql(sqlStatement)

In [73]:
user_df3.show(10)

+-----------+---------------+----------------+---------+-------------+
|total_score|         author|total_decendants|avg_score|total_stories|
+-----------+---------------+----------------+---------+-------------+
|     231445|        vassvdm|          164410|   1129.0|          205|
|     201925|    aarohmankad|           37310|    985.0|          205|
|     149035|       brisance|           66010|    727.0|          205|
|     131200|thereare5lights|           60270|    640.0|          205|
|     123410|         Ours90|           39975|    602.0|          205|
|     100040|            jgv|           12710|    488.0|          205|
|      86305|       riledhel|           39770|    421.0|          205|
|      80155|          dnene|           45305|    391.0|          205|
|      76465|        dralley|           60065|    373.0|          205|
|      69905|           mxgr|           34645|    341.0|          205|
+-----------+---------------+----------------+---------+-------------+
only s

In [88]:
# if there is bias on YC
sqlStatement = """
SELECT score, title,id
FROM `title`
WHERE title like "%Y Combinator%" or title like "%YCombinator%" or title like "%ycombinator%" or title like "%y combinator%"
ORDER BY score  DESC
"""
YC_df = spark.sql(sqlStatement)

In [89]:
YC_df.distinct().show()

+-----+--------------------+-------+
|score|               title|     id|
+-----+--------------------+-------+
|    3|Interview with Ca...|5955210|
|    1|TechStars and YCo...|  14062|
|    1|Rejected from Y C...|6797230|
+-----+--------------------+-------+



In [90]:
# stories of major companies over the year
# apple
sqlStatement = """
SELECT count(*) as total_stories, SUM(score) as total_score,
year(timestamp) as year
FROM title
WHERE title like "%apple%" or title like "%APPLE" or title like "%Apple"
GROUP BY year
ORDER BY year
"""
Comp_df1 = spark.sql(sqlStatement)

In [91]:
Comp_df1.distinct().show()

+-------------+-----------+----+
|total_stories|total_score|year|
+-------------+-----------+----+
|          205|        615|2012|
|          410|       1230|2013|
|          205|        205|2016|
+-------------+-----------+----+

